In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 2 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
Found 3 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-18 19:19 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal data.tsv  /tmp/wordcount/

In [4]:
%%hive

DROP TABLE IF EXISTS data;
DROP TABLE IF EXISTS smallest_values;

DROP TABLE IF EXISTS data;
OK
Time taken: 32.558 seconds
DROP TABLE IF EXISTS smallest_values;
OK
Time taken: 0.015 seconds


In [5]:
%%hive

CREATE TABLE data (letter STRING, dat STRING, numb INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE data;

CREATE TABLE data (letter STRING, dat STRING, numb INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 6.806 seconds
LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE data;
Loading data to table default.data
OK
Time taken: 2.05 seconds


In [7]:
%%hive
CREATE TABLE smallest_values AS
(SELECT DISTINCT numb FROM data ORDER BY numb ASC LIMIT 5);

CREATE TABLE smallest_values AS
(SELECT DISTINCT numb FROM data ORDER BY numb ASC LIMIT 5);
Query ID = root_20200218192324_ffa0a221-9b31-44fc-bc7e-50ba2704dbb8
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582053079980_0003, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0003/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0003
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-18 19:23:33,570 Stage-1 map = 0%,  reduce = 0%
2020-02-18 19:23:40,553 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.25 sec
2020-02-18 19:23:48,488 Stage-

In [8]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM smallest_values;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM smallest_values;
Query ID = root_20200218192419_dcfddb0c-1289-4763-ac1b-7d813f2a321a
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0005, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0005/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0005
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 19:24:31,046 Stage-1 map = 0%,  reduce = 0%
2020-02-18 19:24:36,701 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.3 sec
MapReduce Total cumulative CPU time: 2 seconds 300 msec
Ended Job = job_1582053079980_0005
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-stagi

In [9]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [10]:
!hdfs dfs -cat /tmp/output/000000_0 | head

5
6
7
10
11


In [11]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [12]:
!cat output/000000_0

5
6
7
10
11
